In [1]:
import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

In [2]:
ws = Workspace.from_config()

experiment_name = 'automl-experiment'

experiment = Experiment(ws, experiment_name)

In [3]:
import numpy as np
import pandas as pd
dataset = pd.read_csv("https://raw.githubusercontent.com/AmDeep/Capstone_Project3_AzureML_Nanodegree/main/Data/train.csv")

In [5]:
dataset.head()


,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType

# create a TabularDataset from a delimited file behind a public web url and convert column "Survived" to boolean
web_path_train ='https://raw.githubusercontent.com/AmDeep/Capstone_Project3_AzureML_Nanodegree/main/Data/train.csv'
web_path_test = 'https://raw.githubusercontent.com/AmDeep/Capstone_Project3_AzureML_Nanodegree/main/Data/test.csv'
data_train = Dataset.Tabular.from_delimited_files(path=web_path_train, set_column_types={'Activity': DataType.to_bool()})
data_test = Dataset.Tabular.from_delimited_files(path=web_path_test)

In [7]:
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 10
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=data_train,
                             label_column_name="Activity", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings
                            )

In [10]:
remote_run = experiment.submit(automl_config, show_output = True)


Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_2a22b6cc-37b5-4ba3-ae96-813afe214334

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expe

In [11]:
remote_run


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_2a22b6cc-37b5-4ba3-ae96-813afe214334,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
RunDetails(remote_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [13]:
remote_run.wait_for_completion()


{'runId': 'AutoML_2a22b6cc-37b5-4ba3-ae96-813afe214334',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-09T20:34:51.927019Z',
 'endTimeUtc': '2021-01-09T21:19:18.289976Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '10',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-134141","workspace_name":"quick-starts-ws-134141","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":10,"y_min":null,"y_max":null,"num_classes":n

In [14]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [15]:
best_run


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_2a22b6cc-37b5-4ba3-ae96-813afe214334_27,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
fitted_model


PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [17]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(fitted_model._final_estimator)
print(best_run.get_tags())

Best Run Id:  AutoML_2a22b6cc-37b5-4ba3-ae96-813afe214334_27

 Accuracy: 0.8131191489361702
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('6',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f0d3e4515c0>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                            

In [18]:
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

model_name = best_run.properties['model_name']
model_name

'AutoML2a22b6cc327'

In [19]:
env = best_run.get_environment()

script_file = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

In [20]:
#Register the fitted model
import azureml.train.automl
model = remote_run.register_model(model_name = model_name,
                                  description = 'AutoML model')

In [23]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

aci_service_name = 'activity'
print(aci_service_name)

activity


In [24]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
State: Healthy
Scoring URI: http://0cee9c14-dc60-4977-afff-5580bace1e53.southcentralus.azurecontainer.io/score


In [25]:
#Datapoint 1
%run endpoint.py


{"result": [false, false]}


In [28]:
#Datapoint 2
%run endpoint.py


{"result": [false, false]}


In [27]:
service.get_logs()


'2021-01-09T21:29:35,564630733+00:00 - iot-server/run \n2021-01-09T21:29:35,564784844+00:00 - gunicorn/run \n2021-01-09T21:29:35,564784844+00:00 - rsyslog/run \n2021-01-09T21:29:35,606067734+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [36]:
data_train = data_train.dropna()
ds = data_train.sample(5)
y_actual = ds.pop("Activity")
sample_json = json.dumps({'data':ds.to_dict(orient='records')})
print(sample_json)

{"data": [{"D1": 0.0667, "D2": 0.521209524, "D3": 0.15, "D4": 0.0, "D5": 0.148821083, "D6": 0.653145437, "D7": 0.232076178, "D8": 0.503116911, "D9": 0.730988566, "D10": 0.262403299, "D11": 0.168081487, "D12": 0.0625, "D13": 0.0, "D14": 0.0721, "D15": 0.504902731, "D16": 0.271729049, "D17": 0.601023166, "D18": 0.159148838, "D19": 0.128031386, "D20": 0.0756, "D21": 0.00875, "D22": 0.0, "D23": 1, "D24": 0.0, "D25": 0.546605115, "D26": 0.750976044, "D27": 1, "D28": 1, "D29": 0.0, "D30": 0.27531439, "D31": 0.114282111, "D32": 0.11523484, "D33": 0.0, "D34": 0.0, "D35": 0.0, "D36": 0.0, "D37": 0.0, "D38": 0.295033931, "D39": 0.112614459, "D40": 0.0708, "D41": 0.0, "D42": 0.0, "D43": 0.0, "D44": 0.0, "D45": 0.0, "D46": 0.3344233, "D47": 0.32502861, "D48": 0.0, "D49": 0.0, "D50": 0.83333331, "D51": 0, "D52": 1.0, "D53": 0.00577, "D54": 0.0, "D55": 0.0, "D56": 0.0, "D57": 0.0176, "D58": 0.350492135, "D59": 0.0, "D60": 0.0, "D61": 0.0885, "D62": 0.0, "D63": 0.0753, "D64": 0.0102, "D65": 0.0173, "

In [38]:
results = service.run(sample_json)
print('Predicted Result:- ', results)
print('True Result:- ', y_actual.values)

Predicted Result:-  {"result": [true, false, true, true, true]}
True Result:-  [ True False  True  True  True]


In [39]:
service.delete()